# 使用现有的Index执行RAG

不同搜索模式的效果详细可以参考:https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/azure-ai-search-outperforming-vector-search-with-hybrid/ba-p/3929167


需要将Azure AI Search的Index先创建好，然后再执行下面的步骤。
index 构建可以参考 : https://github.com/radezheng/prepdocs-url


## 导入环境变量
从.env环境里面导入环境变量， 覆盖当前的环境变量.

In [2]:
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryLanguage,
    QueryType,
    RawVectorQuery,
    VectorizableTextQuery,
    VectorFilterMode,    
)

In [3]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv  
import json

load_dotenv(override=True)

service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
model: str = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL") 
credential = AzureKeyCredential(key)

In [4]:
#确认index_name
print(index_name)

gptkbindexrag


In [5]:
# Get the service endpoint and API key from the environment

# Create a client
credential = AzureKeyCredential(key)
client = SearchClient(endpoint=service_endpoint,
                      index_name=index_name,
                      credential=credential)

测试文本搜索

In [18]:
results = client.search(search_text="新型纺纱", top=5, semantic_configuration_name="default",
                        query_type="semantic", search_mode="all")

for result in results:
    print("score:{} : {}: {})".format(result["@search.score"], result["sourcepage"], result["content"]))

score:5.0338974 : 新型纺纱_10946992.pdf#page=9: 高速回转是不合理的。锭子高速必然引起钢丝圈高速,由于钢丝 圈线材截面小,高速回转产生的热量不易散发,容易烧毁,产生飞 圈而造成细纱断头。同时,细纱张力与钢丝圈离心力成正比,而离 心力又与锭速的平方成正比,因此锭速提高,纱线张力也急增而造 成细纱断头。所以,环锭细纱机锭速要进一步提高,会受到钢丝圈 线速度和细纱张力的制约。
(二)气圈稳定性的影响
环锭细纱机在加捻卷绕过程中,因钢丝圈高速回转,在导纱钩 和钢丝圈之间会产生气圈。锭子高速后,使细纱张力及其波动增 大,从而影响气圈的稳定性并增加断头。特别当锭子与筒管的同 心度存在偏差时,因筒管振动而引发锭子振动,严重时会发生“跳 筒管”现象,加剧断头。
可见环锭细纱机要大幅度提高产量,还受到很多不利因素的 限制。因此,各种新型纺纱方法随之问世。
二、新型纺纱的分类与比较
由于环锭纺纱难以在产量上有惊人的突破,说明细纱机要大 幅度提高产量,其方向是在纺纱过程中使加捻与卷绕分开进行,使
1 :unselected: :unselected: :unselected:加捻速度与卷绕速度相互独立而不受牵连。
近几十年来,产生了多种类型的新型纺纱方法,其共同特点是 高速、高产、大卷装、短流程,可直接用条子喂入。所纺纱条大多比 较蓬松,着色性好,有的成纱条干与环锭纱相比毫不逊色。还可充 分利用低级原料、废料和再生纤维,经济效益高。 (一)新型纺纱的分类 新型纺纱的种类很多,就其加捻方法和成纱机理可作如下分 类。
1. 按加捻方法可分为自由端加捻和非自由端加捻两种。自由 端纺纱按纤维凝集和加捻方法不同又可分为转杯纺纱、静电纺纱、 涡流纺纱、摩擦纺纱(Ⅱ 型)、捏锭纺纱、磁性纺纱、搓捻纺纱、液流 纺纱和程控纺纱等。非自由端纺纱按加捻原理可分为自捻纺纱、 无捻纺纱、喷气纺纱、摩擦纺纱(Ⅲ型)以及轴向纺纱等。
2.按成纱机理可分为加捻纺纱、包缠纺纱、无捻纺纱三大类。 包缠纺纱主要有喷气纺纱和平行纺纱等。无捻纺纱有粘合纺纱、 熔融纺纱和缠结纺纱等。
(二)自由端纺纱与非自由端纺纱
1. 自由端纺纱 自由端纺纱是20世纪50 年代逐步发展起来 的新型纺纱方法,其基本特点在于喂入端一定要形成自由端。
自由端的形成,通常采用“断裂”纤维结集体的方法,使喂入端 与加

确认Azure OpenAI 环境变量已经设置好



In [6]:
import os
from openai import AzureOpenAI

chatclient = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2023-05-15"
)

response = chatclient.chat.completions.create(
    model= os.getenv("AZURE_OPENAI_CHAT_DEPLOYED_MODEL"),
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "hi"}
    ]
)

print(response.choices[0].message.content)


Hello! How can I assist you today?


In [19]:
def getKeyword(q):
    response = chatclient.chat.completions.create(
    model= os.getenv("AZURE_OPENAI_CHAT_DEPLOYED_MODEL"),
    messages=[
        {"role": "system", "content": "You are a helpful assistant. please help me to extract the keyword. and response me with the keyword only."},
        {"role": "user", "content": q}
    ]
    )
    return response.choices[0].message.content

定义embedding的方法

In [28]:
emb_model = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL")
openai_client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model=emb_model):
    return openai_client.embeddings.create(input = [text], model=model).data[0].embedding


测试默认搜索下的RAG

In [29]:
#keyword search

q = "员工有几天年假"

results = client.search(search_text=q, top=3)

src = " ###source###: "
for result in results:
    # print("{}: {})".format(result["sourcepage"], result["content"]))
    src += "| sourcepage:" + result["sourcepage"] + " | content:" + result["content"] + " "

metaprompt = "###metaprompt###: you are a helpful assistant. please answer user's question base on the ###source### only, if you don't konw, just say 'no info found', don't make up any unrelevant message. response format: {'sourcepage': '###insert###', 'answer':'###insert###'}"  + src 

msg = [
    {"role": "system", "content": metaprompt},
    {"role": "user", "content": q}
]
display(msg)
print("-------------------------------------------------")
response = chatclient.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYED_MODEL"),
    messages=msg,
    max_tokens=300,
    temperature=0.2,
    top_p=1,
    stop=["###"]
)

#display response with auto wrap
import textwrap
wrapper = textwrap.TextWrapper(width=80)
word_list = wrapper.wrap(text=response.choices[0].message.content)
for element in word_list:
    print(element)
# display(response.choices[0].message.content)

[{'role': 'system',
  'content': "###metaprompt###: you are a helpful assistant. please answer user's question base on the ###source### only, if you don't konw, just say 'no info found', don't make up any unrelevant message. response format: {'sourcepage': '###insert###', 'answer':'###insert###'} ###source###: | sourcepage:EmployeeHandbook_Chinese1107.pdf#page=55 | content: 教育补助计划\n教育补助计划(TAP: Tuition Assistance Program)为符合要求的微软员工提供教育补助经费。每年的补助 经费不超过 20,000 人民币。当年未用完经费不能转入下一年。在提交教育补助计划申请时,员工在公司作\n55 微软 Confidential\n2023/11/01为正式员工入职后应至少工作了一年,且年度绩效评估等级应不为零,并在学习过程中继续保持年度绩效评 估等级不为零。原则上,申请者须保证学习不影响当前工作。\n员工参加的课程应与微软的工作相关,而且必须是学位教育,其内容包括技术、专业或管理能力,学位包括 学士、硕士和博士。非学位的教育培训课程,即教育培训结束无学位授予的情形,不享受 TAP 的支持。 提供课程的教育培训机构须经学校或学院政府协会组织正式批准。员工报销学费的前提条件是所学课程须达 到百分制60分或以上;不以分数考核的课程,需有通过证书。\n员工按照规定的流程提交教育补助计划申请,经批准获得享受TAP 的资格后,可于每年12月和次年 1月根 据报销通知准备报销材料,并在通知的报销窗口期间提交报销申请。\n政策及申请手续的详细内容请查阅: https://microsoft.sharepoint.com/sites/HR web/SitePages/tuitionassistanceprogramCN.aspx\n8. 各类假期 公休假日\n周六、周日为公休假日,员工

-------------------------------------------------
{'sourcepage': 'EmployeeHandbook_Chinese1107.pdf#page=56', 'answer':
'每个员工每个日历年享有年假15天。'}


定义RAG的方法， 根据搜索完的results

In [30]:
def ask(q, results, log=False):
    src = " ###source###: "
    for result in results:
        if log:
            print("ask===={}: {})".format(result["sourcepage"], result["content"]))
        src += "| sourcepage:" + result["sourcepage"] + " | content:" + result["content"] + " "

    metaprompt = "###metaprompt###: you are a helpful assistant. please answer user's question base on the 'content' in ###source### section only, if you don't konw, just say 'no info found', don't make up any unrelevant message. response format: {\"sourcepage\": \"###insert###\", \"answer\":\"###insert###\"}"  + src 

    msg = [
        {"role": "system", "content": metaprompt},
        {"role": "user", "content": q}
    ]
    if log:
        display(msg)
        print("-------------------------------------------------")
    response = chatclient.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYED_MODEL"),
        messages=msg,
        max_tokens=1200,
        temperature=0.2,
        top_p=1,
        stop=["###"]
    )

    # #display response with auto wrap
    # import textwrap
    # wrapper = textwrap.TextWrapper(width=80)
    # word_list = wrapper.wrap(text=response.choices[0].message.content)
    # for element in word_list:
    #     print(element)
    
    #convert response to json

    response_json = json.loads(response.choices[0].message.content)
    return response_json

纯向量搜索

In [32]:
# Pure Vector Search
query = "员工有几天年假"  

mod_query = query
mod_query = getKeyword(query)

print(mod_query)
search_client = SearchClient(service_endpoint, index_name, credential=credential)
# vector_query = VectorizableTextQuery(text=query, k=1, fields="embedding", exhaustive=True)
# Use the below query to pass in the raw vector query instead of the query vectorization  exhaustive=True
vector_query = RawVectorQuery(vector=generate_embeddings(mod_query), k=8, fields="embedding", )
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["sourcepage", "sourcefile", "content"],
    top=1
)  
  
# for result in results:  
#     print(f"sourcepage: {result['sourcepage']}")  
#     print(f"sourcefile: {result['sourcefile']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"Content: {result['content']}")   

r = ask(q=query, results=results)
print(r.get("sourcepage"))
print(r.get("answer"))

年假
EmployeeHandbook_Chinese1107.pdf#page=56
每个员工均享有年假 15天。


文本搜索

In [33]:

#文本搜索
q = "有几天年假"

results = client.search(search_text=q, top=5)

r = ask(q=query, results=results)
display(r)
# print(r.get("sourcepage"))
# print(r.get("answer"))

{'sourcepage': 'EmployeeHandbook_Chinese1107.pdf#page=56',
 'answer': '自每个日历年开始,每个员工均享有年假 15天。'}

混合搜索

In [34]:
# Hybrid Search
query = "有几天年假， 工作8年的员工呢"  
display(query)
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
# vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
vector_query = RawVectorQuery(vector=generate_embeddings(query), k=8, fields="embedding")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["sourcepage", "sourcefile", "content"],
    top=5
)  
  
# for result in results:  
#     print(f"sourcepage: {result['sourcepage']}")  
#     print(f"sourcefile: {result['sourcefile']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"Content: {result['content']}")   

ask(q=query, results=results)

'有几天年假， 工作8年的员工呢'

{'sourcepage': 'EmployeeHandbook_Chinese1107.pdf#page=56', 'answer': '15天'}

混合搜索加语义排名

In [37]:
# Hybrid Search
query = "工作8年的员工有几天年假"  
# query = "出差报销额度是多少"
display(query)
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
# vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
vector_query = RawVectorQuery(vector=generate_embeddings(query), k=5, fields="embedding")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["sourcepage", "sourcefile", "content"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='default', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=8
)  
  
# for result in results:  
#     print(f"sourcepage: {result['sourcepage']}")  
#     print(f"sourcefile: {result['sourcefile']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"Content: {result['content']}")   

ask(q=query, results=results)

    


'工作8年的员工有几天年假'

{'sourcepage': 'EmployeeHandbook_Chinese1107.pdf#page=56', 'answer': '15'}